In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyomo.opt import SolverFactory, TerminationCondition
from src.analyses.value_function import ValueFunction
from src.analyses.convergence_analysis import HausdorffAnalyzer
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import interact
from tqdm.notebook import tqdm
from src.models.cz_model import CaoZavalaModel, CaoZavalaAlgo
from scipy.optimize import curve_fit
def get_critical_width(alg,y_optimal,k=1,abs_epsilon=0.01):
    haus=HausdorffAnalyzer(alg)
    print(f"width: {abs_epsilon/k}")
    print(haus._gen_interval(y_optimal,abs_epsilon/k))
def linear_func(x, a, b):
        return a * x + b
def convergence_analysis(alg,fixed_pt,optimal_sol,start=-5,stop=0,steps=6,name="instance"):

    haus=HausdorffAnalyzer(alg)
    eps_list, distances=haus.analyze(y=fixed_pt, v=optimal_sol,
                    eps_min= start, eps_max = stop, steps= steps,
                    y_optimal= True)
    log_eps = np.log10(eps_list)
    log_distances = np.log10(distances)

    # Perform curve fitting
    params, _ = curve_fit(linear_func, log_eps, log_distances)
    print(f"Slope:{params[0]} intercept:{params[1]}")
    plt.loglog(eps_list, distances)
    plt.grid()
    plt.xticks(np.logspace(start, stop, steps))
    plt.xlabel("epsilon")
    plt.ylabel("Hausdorff distance")
    plt.title(name)
    plt.show()

# List of function names to be used as options
function_names = ['FLECCS', 'poutil', 'ex8_4_4',"process","pooling","crude_selection","tanksize"]
plt.style.use(['./src/utility/' + i + '.mplstyle' for i in ['font-sans', 'size-4-4', 'fontsize']])

# Define your functions here
def FLECCS():
    from NSPLIB.src.instances.FLECCS.src.pyomo_model.model import const_model
    fleccs_obj = -4.946928843629e+08  # Replace with actual value
    fleccs_sol = {'x_sorbent_total': 0.7571021770410511,'x_air_adsorb_max': 0.7567643358197031} 
    fleccs_y_bound = {'x_sorbent_total': [0.0, 1.0],"x_air_adsorb_max": [0.0, 1.0]}

    n_day = 7
    week_diff = 52
    sto_m = const_model(n_day=n_day, week_diff=week_diff)
    m = CaoZavalaModel.from_sto_m(sto_m)
    m.build()
    alg=CaoZavalaAlgo(m,solver="baron")
    convergence_analysis(alg,fleccs_sol,fleccs_obj,
                        start=-5,stop=0,steps=6,name="FLECCS")
    plt.show()
    print("FLECCS executed")

def function2():
    print("Function 2 executed")

def function3():
    print("Function 3 executed")

# Dropdown widget
dropdown = widgets.Dropdown(
    options=function_names,
    description='Select Function:',
    value=function_names[0]
)

output = widgets.Output()

def on_dropdown_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output()
            # Call the function by name
            globals()[change['new']]()

dropdown.observe(on_dropdown_change)

display(dropdown, output)